## Deploy a Machine Learning Model

Let's refactor our churn model from the notebook [02](https://github.com/anferben/machine_learning_zc/blob/main/notebooks/02-classification.ipynb) and [03](https://github.com/anferben/machine_learning_zc/blob/main/notebooks/03-evaluation-metrics-class.ipynb):

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for column in categorical_columns:
    df[column] = df[column].str.lower().str.replace(' ', '_')

df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
df['totalcharges'] = df['totalcharges'].fillna(0)

df['churn'] = (df['churn'] == 'yes').astype('int')

In [3]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [4]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
    'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity','onlinebackup',
    'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract',
    'paperlessbilling', 'paymentmethod']

In [5]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)

    return dv, model

In [6]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [7]:
C=1.0
n_splits = 5

In [8]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train['churn'].values
    y_val = df_val['churn'].values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.840 +- 0.008


In [9]:
scores

[0.8423907497389926,
 0.8458834094821879,
 0.8311780052177403,
 0.8301684306452645,
 0.8509263907627623]

In [12]:
dv, model = train(df_full_train, df_full_train['churn'].values, C=C)
y_pred = predict(df_test, dv, model)

y_test = df_test['churn'].values
auc = roc_auc_score(y_test, y_pred)
auc

0.8572386167896259

We can not use a jupyter notebook to put it in a web service, we need a python script. Firstly we're going to save this as a `pickle`:

In [13]:
import pickle

In [14]:
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

We are going to use `'wb'` in the `open` function to indicate that we are going to write binary not text:

In [ ]:
f_out = open(output_file, 'wb')
pickle.dump((dv, model), f_out)
f_out.close()

Remember the importance to close the file, in order to not have later problems. To make sure the file is close, we can do the same process with this structure:

In [15]:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

Let's pretend now that we are loading the file from anywhere else, by __restarting the kernel__ here.

In [1]:
import pickle

Reading the file:

In [4]:
output_file = f'model_C=1.0.bin'

In [5]:
with open(output_file, 'rb') as f_in:
    (dv, model) = pickle.load(f_in)

In [6]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

Creating a customer to use the model:

In [15]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month_to_month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [16]:
X = dv.transform([customer])

In [17]:
model.predict_proba(X)[0, 1]

0.5724415801058189

In order to have this notebook into a python file, just go to the top menu on the notebook, click the the three points and click on `Export`; select `as python script`.